## Trademo Assignment

In [1]:
#let's install the required libraries
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.9 MB/s eta 0:00:00


In [2]:
!pip install llama-index==0.9.40

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [3]:
#Installing a library to create embedding
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.1 MB/s eta 0:00:00


In [4]:
#Importing requiring libraries
from llama_index import VectorStoreIndex,ServiceContext
from llama_index import download_loader
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [5]:
SimpleCSVReader = download_loader("SimpleCSVReader")
loader = SimpleCSVReader(encoding="utf-8")
documents = loader.load_data('/content/trademo_data/trademo_data.csv')

In [6]:
system_prompt = """ You are an expert Q&A chatbot with deep knowledge of the data provided to you.Your goal is to answer the question as accurately as possible based on the instructions and context provided.
If you don't know the answer than just say that you don't know the answer, but don't provide any wrong information"""

#default format suitable by Llama2
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|Assistant|>")

In [7]:
#Initiating hugging face CLI
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [8]:
#Using Hugging face model here
import torch
llm = HuggingFaceLLM(context_window = 4096,
max_new_tokens = 1024,
generate_kwargs = {"temperature": 0.0, "do_sample": False},
system_prompt = system_prompt,
query_wrapper_prompt = query_wrapper_prompt,
tokenizer_name = "meta-llama/Llama-2-7b-chat-hf",
model_name = "meta-llama/Llama-2-7b-chat-hf",
device_map = "auto",
#uncomment this if using CUDA to reduce memory usage
model_kwargs = {"torch_dtype": torch.float16, "load_in_8bit": True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
#creating my service context for further generation of responses from chatbot
service_context = ServiceContext.from_defaults(
    chunk_size = 1024,
    llm = llm,
    embed_model = embed_model)

In [11]:
#Displaying my service context
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=1024, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7d3df449d900>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7d3df449d900>, id_func=<function default_id_func at 0x7d3e9f89b5b0>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.logger.base.LlamaLogger object at 0x7d3ed97fd840>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7d3df449d900>)

In [12]:
#creating my index here
index = VectorStoreIndex.from_documents(documents, service_context= service_context)

In [13]:
index

In [14]:
#creating my query engine
query_engine = index.as_query_engine()

In [15]:
response = query_engine.query("Tell me the port with highest TEU in terms of Total TEU and Change in percentage of TEU")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [16]:
print(response)

Based on the information provided, the port with the highest TEU in terms of Total TEU is YANTIAN, CHINA with 10309263.38 TEUs.

As for the change in percentage of TEU, the port with the highest percentage change is LOS ANGELES, CALIFORNIA with a percentage change of -52.84.

Please let me know if you have any further questions.


In [17]:
response2 = query_engine.query("Tell me the top 10 port with highest TEU in terms of Total TEU and Change in percentage of TEU")

In [18]:
print(response2)

Based on the data provided, the top 10 ports with the highest TEU are:

1. YANTIAN, CHINA - 10309263.38 TEU (31.18% change in TEU)
2. DUBAI, PORT RASHID, UNITED ARAB EMIRATES - 35 TEU (4.71% change in TEU)
3. ALL OTHER JORDAN PORTS, JORDAN - 251.31 TEU (35.07% change in TEU)
4. ALL OTHER JORDAN PORTS, JORDAN - 202 TEU (28.19% change in TEU)
5. WILMINGTON, NORTH CAROLINA - 202 TEU (28.19% change in TEU)
6. CHARLESTON, SOUTH CAROLINA - 84 TEU (11.72% change in TEU)
7. NEW YORK/NEWARK AREA, NEW JERSEY - 4 TEU (83.33% change in TEU)
8. LOS ANGELES, CALIFORNIA - 90888.93 TEU (52.84% change in TEU)
9. SAN FRANCISCO, CALIFORNIA - 92975.63 TEU (49.5% change in TEU)
10. HOUSTON, TEXAS - 113130.86 TEU (33.33% change in TEU)

Note: The change in percentage of TEU is calculated by dividing the difference in TEU between two time periods by the TEU in the earlier time period, and multiplying by 100.


In [23]:
response3 = query_engine.query("Give me top 20 laiding and Unlaiding Ports in terms of overall percentage change in TEU")

In [24]:
print(response3)

Based on the information provided, the top 20 leading and unlaiding ports in terms of overall percentage change in TEU are:

Leading Ports:

1. YANTIAN, CHINA - 31.18%
2. CAPELLE, CAPELLE AAN DEN IJSSEL, DEN HELDER, GRONINGEN, HAARLEM, HAREN, HARLINGEN, HELDER, HOOK OF HOLLAND, MIDDLEBURG, SAS VAN GENT, SCHIEDAM, ZAANDAM - 19.14%
3. SAN JUAN, PUERTO RICO - 100%

Unlaiding Ports:

1. FREEPORT, TEXAS - 206, 14.37%, -100%
2. PORTLAND, MAINE - 72, 5.02%, 0%

Note: The percentage change in TEU is calculated based on the total TEU of each port in the given time period compared to the previous time period.
